Copyright 2022 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title License
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Shortcut testing with synthetic images

We illustrate the method proposed in Brown et al. 2022 (https://arxiv.org/abs/2207.10384) to detect model shortcutting when a model relies on both signals related to the label Y and to an auxiliary/sensitive attribute A.

To demonstrate ShorT, we need a dataset such that the main (Y) and auxiliary (A) tasks are not too easy and "help" each other. We refer to the MNIST dataset, with the aim to discriminate between numbers smaller than 5 and larger than 5. We add a confounder to each image, i.e. a small colored squared. The color of the square (red or green) can then be correlated to the label in the image to create a spurious signal. We add significnat amounts of noise to both signals to avoid a "binary" behavior of the model (i.e. when it fully relies on one signal or the other, but never on both).

We demonstrate how ShorT identifies the confounding, when present.

# Imports

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import importlib
import random

from typing import *

import attr
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import numpy as np
import seaborn as sns
import scipy

from matplotlib.cm import get_cmap
from matplotlib import gridspec
from matplotlib import pyplot as plt
from matplotlib import cm

import sklearn
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import numpy as np


In [ ]:
!pip install ml_collections

In [ ]:
import ml_collections as mlc

# Synthetic dataset from MNIST
Images are generated by adding a red or green square on an MNIST image ([Deng et al., 2012](https://ieeexplore.ieee.org/document/6296535)). We select our labels to be an image with a number smaller than 5 (`label=0`) or larger than 5 (`label=1`). Two parameters specify the proportion (in %) of each label being red. A larger number (> 50%) means that the class label will be correlated with the red color, while a smaller number (< 50%) means that the class label will be correlated with the green color.

Refer to the config dictionary (`cfg_data`) to specify all the data generation parameters. Please note that it is important to check that the tasks (predicting the label and predicting the attribute) are not trivial or too simple.

In [ ]:
#@title Data utils

def add_square_to_image(image, color, square_size, noise_thresh = 0.1):
  image_size = image.shape
  start_x = np.random.choice(image_size[0]-square_size)
  end_x = min(image_size[0], start_x+square_size)
  start_y = np.random.choice(image_size[1]-square_size)
  end_y = min(image_size[1], start_y+square_size)
  square = np.random.uniform(
      size=(square_size, square_size, image_size[-1])) >= noise_thresh
  image[start_x:end_x, start_y:end_y, :] = square * np.array(color) #color
  return image, square, start_x, start_y

def create_images_dataset(corr_a_y0, corr_a_y1, n_images_per_class, n_labels,
                          select_labels, n_colors, colors, color_map,
                          noise_thresh, conf_noise=0.1, mnist_split="train"):

  cls0_corr = float(corr_a_y0) / 100
  cls1_corr = float(corr_a_y1) / 100

  conprob_colors_orie = np.array([[cls0_corr, 1 - cls0_corr],
                                  [cls1_corr, 1 - cls1_corr]])
  
  
  # Load MNIST
  (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
  if mnist_split == "train":
    images = train_images
    labels = train_labels
  else:
    images = test_images
    labels = test_labels
  
  n_images = n_images_per_class * n_labels
  n_pixels = images[0].shape[0]
  images_colored = np.zeros((n_images, n_pixels, n_pixels, 3))
  flip_images = np.zeros((n_images, n_pixels, n_pixels, 3))
  image_labels = np.zeros((n_images, 1), dtype=int)
  color_labels = np.zeros((n_images, 1), dtype=int)

  for i, label in enumerate(select_labels):
    selected = labels == label[0]
    j = 1
    while j<len(label):
      selected = np.logical_or(selected, labels == label[j])
      j += 1
    selected_label = labels[selected]
    selected_images = images[selected, :,:]
    for class_image_index in range(n_images_per_class):
      image_index = i * n_images_per_class + class_image_index
      color = np.random.choice(n_colors, p=conprob_colors_orie[i])
      flip_color = 1 if color == 0 else 0
      # randomly select image from MNIST
      idx = np.random.choice(selected_label.shape[0], replace=True)
      image = np.reshape(selected_images[idx,:,:], (n_pixels,n_pixels, 1))
      channels = np.concatenate((image, image, image), axis=2)
      # Add colored square to the image
      images_colored[image_index], square, x, y = add_square_to_image(channels/255,
                                                        color_map[colors[color]],
                                                        int(n_pixels/7),
                                                        noise_thresh=conf_noise)
      flip_images[image_index] = channels/255
      sq_szx, sq_szy, _ = square.shape
      flip_images[image_index,x:x+sq_szx,y:y+sq_szy, :] = square * color_map[colors[flip_color]]

      noise_img = np.random.uniform(
          size=[n_pixels, n_pixels, 1]) < noise_thresh  # white noise
      noise_img = np.concatenate((noise_img, noise_img, noise_img), axis=2)
      noise_img =  noise_img.astype('float64')
      noise_img[x:x+sq_szx,y:y+sq_szy, :] = square * (0.,0.,0.) # remove noise from added square

      images_colored[image_index] = np.clip(
          images_colored[image_index] + noise_img, 0.0, 1.0)
      flip_images[image_index] = np.clip(
          flip_images[image_index] + noise_img, 0.0, 1.0)

      image_labels[image_index,:] = i
      color_labels[image_index,:] = color

  return images_colored, image_labels, color_labels, flip_images


def gen_dataset(cfg_data):
  """Generates bars dataset.

  Each image contains a single bar which is either horizontal (class 0) or
  vertical (class 1)
  The bar is either red (concept 0) or green (concept 1)
  """

  tr_im, tr_lab, tr_col_lab, tr_fl_im = create_images_dataset(**cfg_data)
  
  train_images, train_labels, train_color_labels, train_flip_images = \
  sklearn.utils.shuffle(tr_im, tr_lab, tr_col_lab, tr_fl_im)

  dataset = dict(
      train=dict(
          image=train_images,
          label=train_labels,
          color_label=train_color_labels,
          flip_image=train_flip_images),
  )
  return dataset

In [ ]:
#@title Synthetic data parameters

cfg_data = mlc.ConfigDict()

# These must show a large difference to encourage shortcutting
cfg_data.corr_a_y0 = 20  #@param Proportion of images with label 0 being red
cfg_data.corr_a_y1 = 95  #@param Proportion of images with label 1 being red

# Note: asymmetrical, high correlations (i.e. corr_y0 + corr_y1 != 100) lead to 
# unfairness in terms equalized odds

cfg_data.n_images_per_class = 5000  #@param

cfg_data.noise_thresh = 0.5  # noise added to the number
cfg_data.conf_noise = 0.6    # noise added to the confounder

cfg_data.colors = ['red', 'green', 'blue']
cfg_data.color_map = mlc.ConfigDict()
cfg_data.color_map.red = (1., 0., 0.)
cfg_data.color_map.green = (0., 1., 0.)
cfg_data.color_map.blue = (0., 0., 1.)

cfg_data.n_colors = 2
cfg_data.n_labels = 2

cfg_data.select_labels = [[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]
label_names = ['<5', '>=5']
cfg_data.mnist_split = "train"

# set the seed
seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)

Based on these parameters, we generate train, validation and test data. In addition, we can generate "counterfactuals" (referred to as 'flipped_images') that flip the sampled color label before generating the same image.

In [ ]:
#@title Train and validation data
dataset = gen_dataset(cfg_data)

In [ ]:
#@markdown Explore the dataset
dataset["train"].keys()

In [ ]:
#@markdown Compute proportions of the data to check specifications
# The output numbers should approximate the desired correlations between Y and A

n_r1 = np.sum(np.logical_and(dataset["train"]["label"] == 0, 
                             dataset["train"]["color_label"] == 0)) / np.sum(dataset["train"]["label"] == 0)
print("Proportion of 0 labels that are red: {}".format(n_r1))

n_r1 = np.sum(np.logical_and(dataset["train"]["label"] == 1,
                             dataset["train"]["color_label"] == 0)) / np.sum(dataset["train"]["label"] == 1)
print("Proportion of 1 labels that are red: {}".format(n_r1))

In [ ]:
#@markdown Visualize the data (5 random samples)

split = "train"
NUM_TEST_SAMPLES = len(dataset[split]["label"])
print(NUM_TEST_SAMPLES, dataset[split]["label"])

for idx in random.choices(range(NUM_TEST_SAMPLES), k=5):
  plt.figure()
  print(dataset[split]["label"][idx], dataset[split]["color_label"][idx])
  plt.imshow(dataset[split]["image"][idx, ...])
  plt.show()
n_pixels, n_pixels, n_channels = dataset[split]["image"][idx, ...].shape

In [ ]:
#@title Test data

cfg_data.n_images_per_class = 2000  #@param
cfg_data.mnist_split = "test"  # We select images from the "test" MNIST split

test_dataset = gen_dataset(cfg_data)

In [ ]:
#@title Transform data into TF Datasets
def tf_data(dataset, batch_size = 64, valid_prop = 0.2, is_train = True):

  # Original images
  split = "train"
  training_data = tf.data.Dataset.from_tensor_slices((dataset[split]["image"],
                              dataset[split]["label"],
                              dataset[split]["color_label"]))
  n_images = dataset[split]["label"].shape[0]
  n_valid = math.floor(n_images * valid_prop)
  valid_data = training_data.take(n_valid).cache().repeat().batch(batch_size)
  ds = training_data.skip(n_valid).take(n_images - n_valid).cache()
  # Flipped color on the same images
  data_flipped = tf.data.Dataset.from_tensor_slices((dataset[split]["flip_image"],
                              dataset[split]["label"],
                              np.logical_not(dataset[split]["color_label"]).astype(float)))
  
  if is_train:
    # Cannot be used as pairs due to shuffling
    data = ds.repeat().shuffle(10000).batch(batch_size)
    data_flipped = data_flipped.repeat().shuffle(10000).batch(batch_size)
  else:
    # Can be used for paired comparisons
    data = ds.repeat().batch(batch_size)
    data_flipped = data_flipped.repeat().batch(batch_size)
  
  return data, data_flipped, valid_data

In [ ]:
pos = mlc.ConfigDict()
pos.x, pos.y, pos.a = 0, 1, 2  # Associates a position in the tf dataset to a variable

# Train and validation data
train_data, _, valid_data = tf_data(dataset, 64)

In [ ]:
# Test data - avoinding to shuffle so we can compare the images and their counterfactuals
test_data, test_data_flipped, _ = tf_data(test_dataset, 64, valid_prop=0.0, is_train=False)

# Models

## Baseline model - SingleHead class
We build simple MLP models that predict one binary label. This model can be used to assess the performance of the baseline model f(X) -> Y.

## Attribute encoding - SingleHead with frozen feature extractor
In addition, this model can take as an extra input a frozen feature extractor. It then adds a linear layer to this feature extractor to perform the binary classification. Only the weights of this layer are tuned during training. This architecture is used to assess the level of attribute encoding of another model.

## Multi-task - MultiHead with Gradient Reversal
The architecture of the multi-task model first comprises a series of layers that represent a "feature extractor". On top of the feature extractor, we add two heads: 
- one head for the label, which is predicted from a single linear layer,
- one head for an auxiliary task (here the prediction of the color of the square). This head includes at least a non-linear layer to allow a gradient scaling operation.

The weight of the auxiliary loss in the total loss is a hyper-parameter in the method (weight of label loss is fixed to 1.0). A gradient reversal head is added to the auxiliary task. Gradient scaling can then be performed (positive or negative, extra hyper-parameter). Setting both hyper-parameters to 0 corresponds to the baseline single head model.


A config (`cfg`) dictionary specifies the parameters of the MLP architecture.

In [ ]:
#@title Model utils

class GradientReversal(tf.keras.layers.Layer):

    @tf.custom_gradient
    def grad_reverse(self, x):
        y = tf.identity(x)
        def custom_grad(dy):
          return self.hp_lambda * dy
        return y, custom_grad

    def __init__(self, hp_lambda, **kwargs):
        super(GradientReversal, self).__init__(**kwargs)
        self.hp_lambda = K.variable(hp_lambda, dtype='float', name='hp_lambda')

    def call(self, x, mask=None):
        return self.grad_reverse(x)

    def set_hp_lambda(self,hp_lambda):
        K.set_value(self.hp_lambda, hp_lambda)
    
    def increment_hp_lambda_by(self,increment):
        new_value = float(K.get_value(self.hp_lambda)) +  increment
        K.set_value(self.hp_lambda, new_value)

    def get_hp_lambda(self):
        return float(K.get_value(self.hp_lambda))


class BaselineArch():
  """Superclass for multihead training."""

  def __init__(self, main="y", aux=None, dtype=tf.float32, pos=None):
    """Initializer.

    Args:
      main: name of variable for the main task
      aux: nema of the variable for the auxiliary task
      dtype: desired dtype (e.g. tf.float32).
      pos: ConfigDict that specifies the index of x, y, c, w, u in data tuple.
        Default: data is of the form (x, y, c, w, u).
    """
    self.model = None
    self.inputs = "x"
    self.main = main
    self.aux = aux
    self.dtype = dtype
    if pos is None:
      pos = mlc.ConfigDict()
      pos.x, pos.y, pos.a = 0, 1, 2
    self.pos = pos

  def get_input(self, *batch):
    """Fetch model input from the batch."""
    # first input
    stack = tf.cast(batch[self.pos[self.inputs[0]]], self.dtype)
    # fetch remaining ones
    for c in self.inputs[1:]:
      stack = tf.concat([stack, tf.cast(batch[self.pos[c]], self.dtype)],
                        axis=1)
    return stack

  def get_output(self, *batch):
    """Fetch outputs from the batch."""
    if self.aux:
      return (tf.cast(batch[self.pos[self.main]],self.dtype),
              tf.cast(batch[self.pos[self.aux]], self.dtype))
    else:
      return (tf.cast(batch[self.pos[self.main]],self.dtype))

  def split_batch(self, *batch):
    """Split batch into input and output."""
    return self.get_input(*batch), self.get_output(*batch)

  def fit(self, data: tf.data.Dataset, **kwargs):
    """Fit model on data."""
    ds = data.map(self.split_batch)
    self.model.fit(ds, **kwargs)

  def predict(self, model_input, **kwargs):
    """Predict target Y given the model input. See also: predict_mult()."""
    y_pred = self.model.predict(model_input, **kwargs)
    return y_pred

  def predict_mult(self, data: tf.data.Dataset, num_batches: int, **kwargs):
    """Predict target Y from the TF dataset directly. See also: predict()."""
    y_true = []
    y_pred = []
    ds_iter = iter(data)
    for _ in range(num_batches):
      batch = next(ds_iter)
      model_input, y = self.split_batch(*batch)
      y_true.extend(y)
      y_pred.extend(self.predict(model_input, **kwargs))
    return np.array(y_true), np.array(y_pred)

  def score(self, data: tf.data.Dataset, num_batches: int, 
            metric: tf.keras.metrics.Metric , **kwargs):
    """Evaluate model on data.

    Args:
      data: TF dataset.
      num_batches: number of batches fetched from the dataset.
      metric: which metric to evaluate (schrouf not be instantiated).
      **kwargs: arguments passed to predict() method.

    Returns:
      score: evaluation score.
    """
    y_true, y_pred = self.predict_mult(data, num_batches, **kwargs)
    return metric()(y_true, y_pred).numpy()


class MultiHead(BaselineArch):
  """Multihead training."""

  def __init__(self, cfg, main, aux, dtype=tf.float32, pos=None): 
    """Initializer.

    Args:
      cfg: A config that describes the MLP architecture.
      main: variable for the main task
      aux: variable for the auxialiary task
      dtype: desired dtype (e.g. tf.float32) for casting data.
    """
    super(MultiHead, self).__init__(main, aux, dtype, pos)
    self.main = "y"
    self.aux = "a"
    self.cfg = cfg
    # build architecture
    self.model, self.feat_extract = self.build()

  def build(self):
    """Build model."""
    cfg = self.cfg
    input_shape = cfg.model.x_dim

    # set config params to defaults if missing
    use_bias = cfg.model.get("use_bias", True)
    activation = cfg.model.get("activation", "relu")
    output_activation = cfg.model.get("output_activation", "sigmoid")

    model_input = tf.keras.Input(shape=input_shape)
    flatten_input = tf.keras.layers.Flatten()(model_input)
    if cfg.model.depth:
      x = tf.keras.layers.Dense(cfg.model.width, use_bias=use_bias,
                                activation=activation,
                                kernel_regularizer=cfg.model.regularizer)(flatten_input)
      for _ in range(cfg.model.depth - 1):
        x = tf.keras.layers.Dense(cfg.model.width, use_bias=use_bias,
                                  activation=activation,
                                  kernel_regularizer=cfg.model.regularizer)(x)
    else:
      x = flatten_input
    feature_extractor = tf.keras.models.Model(inputs=flatten_input,
                                              outputs=x)
    # output layer - a single linear layer
    y = tf.keras.layers.Dense(cfg.model.output_dim,
                              use_bias=cfg.model.use_bias,
                              name="output",
                              activation=output_activation,
                              kernel_regularizer=cfg.model.regularizer)(x)
    # attribute layer - an extra dense layer is required for gradients to flow back
    attr_activation = cfg.model.get("attr_activation", "sigmoid")
    input_branch_a = GradientReversal(hp_lambda=cfg.model.attr_grad_updates)(x)
    a_branch = tf.keras.layers.Dense(cfg.model.branch_dim,
                    use_bias=cfg.model.use_bias,
                    name="attr_branch",
                    activation=activation,
                    kernel_regularizer=cfg.model.regularizer)(input_branch_a)
    a = tf.keras.layers.Dense(cfg.model.attr_dim,
                        use_bias=cfg.model.use_bias,
                        name="attribute",
                        activation=attr_activation,
                        kernel_regularizer=cfg.model.regularizer)(a_branch)
    


    # choose optimizer
    if cfg.opt.name == "sgd":
      opt = tf.keras.optimizers.SGD(learning_rate=cfg.opt.learning_rate,
                                    momentum=cfg.opt.get("momentum", 0.9))
    elif cfg.opt.name == "adam":
      opt = tf.keras.optimizers.Adam(learning_rate=cfg.opt.learning_rate)
    else:
      raise ValueError("Unrecognized optimizer type."
                       "Please select either 'sgd' or 'adam'.")

    # define losses
    losses = {
        "output": cfg.model.get("output_loss", "binary_crossentropy"),
        "attribute": cfg.model.get("attribute_loss", "binary_crossentropy")
    }
    loss_weights = {"output": 1.0,
                    "attribute": cfg.get("attr_loss_weight", 1.0)}
    metrics = {"output": tf.keras.metrics.AUC(),
               "attribute": tf.keras.metrics.AUC()}

    # build model
    model = tf.keras.models.Model(inputs=model_input, outputs=[y,a])
    model.build(input_shape)
    # model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(),
    #               metrics=tf.keras.metrics.BinaryAccuracy())
    model.compile(optimizer=opt, loss=losses, loss_weights=loss_weights,
                  metrics=metrics)
    return model, feature_extractor

  def predict_mult(self, data: tf.data.Dataset, num_batches: int, **kwargs):
    """Predict from the TF dataset directly. See also: predict()."""
    # infer dimensions
    pos = self.pos
    batch = next(iter(data))
    y_dim = batch[pos.y].shape[1]
    a_dim = batch[pos.a].shape[1]

    # begin
    data_iter = iter(data)
    a_true_all = np.array([]).reshape((0, a_dim))
    a_pred_all = np.array([]).reshape((0, a_dim))
    y_true_all = np.array([]).reshape((0, y_dim))
    y_pred_all = np.array([]).reshape((0, y_dim))

    for _ in range(num_batches):
      batch = next(data_iter)
      x, y_true, a_true = batch[pos.x], batch[pos.y], batch[pos.a]
      y_pred, a_pred = self.predict(x, **kwargs)
      a_true_all = np.append(a_true_all, a_true, axis=0)
      a_pred_all = np.append(a_pred_all, a_pred, axis=0)
      y_true_all = np.append(y_true_all, y_true, axis=0)
      y_pred_all = np.append(y_pred_all, y_pred, axis=0)

    return (y_true_all, a_true_all), (y_pred_all, a_pred_all)

  def score(self, data: tf.data.Dataset, num_batches: int, 
            metric: tf.keras.metrics.Metric, **kwargs):
    """Evaluate model on data.

    Args:
      data: TF dataset.
      num_batches: number of batches fetched from the dataset.
      metric: which metric to evaluate (should not be instantiated).
      **kwargs: arguments passed to predict() method.

    Returns:
      score: evaluation score.
    """
    out_true, out_pred = self.predict_mult(data, num_batches, **kwargs)
    scores = []
    for head in range(len(out_true)):
      score = metric()(out_true[head], out_pred[head])
      scores.append(score.numpy())
    return scores


# Can be used as a single task model fully trained or from a pre-trained
# feature extractor

class SingleHead(BaselineArch):
  """Singlehead training."""

  def __init__(self, cfg, main, dtype=tf.float32, pos=None, feat_extract=None): 
    """Initializer.

    Args:
      cfg: A config that describes the MLP architecture.
      main: variable for the main task
      aux: variable for the auxialiary task
      dtype: desired dtype (e.g. tf.float32) for casting data.
    """
    super(SingleHead, self).__init__(main, None, dtype, pos)
    self.main = "a"
    self.cfg = cfg
    # build architecture
    self.model = self.build(feat_extract)

  def build(self, feat_extract=None):
    """Build model."""
    cfg = self.cfg
    input_shape = cfg.model.x_dim

    # set config params to defaults if missing
    use_bias = cfg.model.get("use_bias", True)
    activation = cfg.model.get("activation", "relu")
    output_activation = cfg.model.get("output_activation", "sigmoid")

    model_input = tf.keras.Input(shape=input_shape)
    flatten_input = tf.keras.layers.Flatten()(model_input)
    if not feat_extract:
      if cfg.model.depth:
        x = tf.keras.layers.Dense(cfg.model.width, use_bias=use_bias,
                                  activation=activation,
                                  kernel_regularizer=cfg.model.regularizer)(flatten_input)
        for _ in range(cfg.model.depth - 1):
          x = tf.keras.layers.Dense(cfg.model.width, use_bias=use_bias,
                                    activation=activation,
                                    kernel_regularizer=cfg.model.regularizer)(x)
      else:
        x = flatten_input
      feature_extractor = x
    else:
      feat_extract.trainable = False
      feature_extractor = feat_extract(flatten_input, training=False)
  
    # output layer
    y = tf.keras.layers.Dense(cfg.model.output_dim,
                              use_bias=cfg.model.use_bias,
                              name="output",
                              activation=output_activation,
                              kernel_regularizer=cfg.model.regularizer)(feature_extractor)  

    # choose optimizer
    if cfg.opt.name == "sgd":
      opt = tf.keras.optimizers.SGD(learning_rate=cfg.opt.learning_rate,
                                    momentum=cfg.opt.get("momentum", 0.9))
    elif cfg.opt.name == "adam":
      opt = tf.keras.optimizers.Adam(learning_rate=cfg.opt.learning_rate)
    else:
      raise ValueError("Unrecognized optimizer type."
                       "Please select either 'sgd' or 'adam'.")

    # build model
    model = tf.keras.models.Model(inputs=model_input, outputs=y)
    model.build(input_shape)
    model.compile(optimizer=opt,
                  loss=cfg.model.get("output_loss", "binary_crossentropy"),
                  metrics=tf.keras.metrics.AUC())

    return model

In [ ]:
#@markdown Model parameters 
cfg = mlc.ConfigDict()

cfg.model = mlc.ConfigDict()
cfg.model.width = 10  #@param architecture width
cfg.model.depth = 3  #@param model depth
cfg.model.use_bias = True  # whether we add biases to activations.
cfg.model.activation = 'relu'
cfg.model.x_dim = (n_pixels, n_pixels, n_channels)
cfg.model.branch_dim = 2  # architecture width within each branch
cfg.model.regularizer = None  # replace with e.g. 'l2' for weight decay

# output head
cfg.model.output_activation = 'sigmoid'
cfg.model.output_dim = 1

# attribute head
cfg.model.attr_activation = 'sigmoid'
cfg.model.attr_grad_updates = float(-0.05)
cfg.model.attr_dim = 1
# this is a tradeoff between the loss on A and the loss on target Y.
# If it's zero, we ignore the attribute loss completely.
cfg.attr_loss_weight = float(1.0)

cfg.opt = mlc.ConfigDict()
cfg.opt.name = 'adam'
cfg.opt.learning_rate = 0.001
cfg.opt.momentum = 0.9

# Baseline task model

In [ ]:
metric = tf.keras.metrics.AUC

baseline = []
for seed in [0,1,2,3,4]:
  tf.random.set_seed(seed)
  np.random.seed(seed)
  enc = SingleHead(cfg, main="y", dtype=tf.float32, feat_extract=None)
  kwargs = {'epochs': 100, 'steps_per_epoch':20, 'verbose': False}
  enc.fit(train_data, **kwargs)
  kwargs = {'verbose': False}
  num_batches = 30
  sc = enc.score(valid_data, num_batches, metric, **kwargs)
  baseline.append(sc)

print("Baseline model: %1.2f +- %.2f" % (np.mean(baseline),np.std(baseline)))

# Bounds on attribute encoding (i.e. LEB and UEB in paper)

In [ ]:
#@title Upper bound
metric = tf.keras.metrics.AUC

upper_bound = []
for seed in [0,1,2,3,4]:
  tf.random.set_seed(seed)
  np.random.seed(seed)
  enc = SingleHead(cfg, main="a", dtype=tf.float32, feat_extract=None)
  kwargs = {'epochs': 100, 'steps_per_epoch':20, 'verbose': False}
  enc.fit(train_data, **kwargs)
  kwargs = {'verbose': False}
  num_batches = 30
  sc = enc.score(valid_data, num_batches, metric, **kwargs)
  upper_bound.append(sc)

print("Upper bound: %1.2f +- %.2f" % (np.mean(upper_bound),np.std(upper_bound)))

In [ ]:
#@title Lower bound
# Predict a color randomly selected from the train set (with replacement)

n_train, a_dim = dataset["train"]["color_label"].shape

lower_bound = []
for seed in [0,1,2,3,4]:
  tf.random.set_seed(seed)
  np.random.seed(seed)
  data_iter = iter(train_data)
  a_true_all = np.array([]).reshape((0, a_dim))
  a_pred_all = np.array([]).reshape((0, a_dim))

  for _ in range(num_batches):
    batch = next(data_iter)
    a_true = batch[pos.a]
    a_pred = dataset["train"]["color_label"][np.random.choice(n_train,a_true.shape[0]), ...]
    preds = a_pred.reshape((-1,a_dim))
    a_true_all = np.append(a_true_all, a_true, axis=0)
    a_pred_all = np.append(a_pred_all, preds, axis=0)

  sc = tf.keras.metrics.AUC()(a_true_all, a_pred_all).numpy()
  lower_bound.append(sc)

print("Lower bound: %1.2f +- %.2f" % (np.mean(lower_bound),np.std(lower_bound)))

# Example of Multi-Task model

In [ ]:
#@title Initialize from config
cfg.model.attr_grad_updates = float(0.1) # if both are set to 0, this is a baseline model (i.e. the same as with no extra head)
cfg.attr_loss_weight = float(0.75)
clf = MultiHead(cfg, main="y", aux="a", dtype=tf.float32, pos=None)

In [ ]:
#@markdown Explore the architecture
clf.model.summary()

In [ ]:
#@title Train the model
kwargs = {'epochs': 100, 'steps_per_epoch':20, 'verbose': True}
clf.fit(train_data, **kwargs)
clf.trainable=False

In [ ]:
#@title Define the decision threshold by maximizing the F1-score on validation data

from sklearn.metrics import precision_recall_curve
def f1_curve(truth, prediction_scores, e=1e-6):
  precision, recall, thresholds = precision_recall_curve(truth, prediction_scores)
  f1 = 2*recall*precision/(recall+precision+e)
  return thresholds, f1[:-1]

def threshold_at_max_f1_score(truth, prediction_scores):
  thresholds, f1 = f1_curve(truth, prediction_scores)
  peak_idx = np.argmax(f1)
  return thresholds[peak_idx]

In [ ]:
kwargs = {'verbose': False}
num_batches = 20
metric = tf.keras.metrics.AUC
scores_val = clf.score(valid_data, num_batches, metric, **kwargs)
print(scores_val)

In [ ]:
kwargs = {'verbose': False}
yt, yp = clf.predict_mult(valid_data, num_batches, **kwargs)
threshold = threshold_at_max_f1_score(yt[0],yp[0])
threshold

In [ ]:
#@title Fairness metrics

# As per the work of Alabdulmohsin et al., 2021

def fairness_metrics(y_pred, y_true, sens_attr):
  eps = 1e-5
  groups = np.unique(sens_attr).tolist()

  max_error = 0
  min_error = 1

  max_mean_y = 0
  min_mean_y = 1

  max_mean_y0 = 0  # conditioned on y = 0
  min_mean_y0 = 1

  max_mean_y1 = 0
  min_mean_y1 = 1

  for group in groups:
    yt = y_true[sens_attr == group].astype('int32')
    ypt = (y_pred[sens_attr == group]).astype('int32')
    err = -np.mean(yt * np.log(ypt+eps) + (1-yt)*np.log(1-ypt+eps))
    mean_y = np.mean(y_pred[sens_attr == group])
    neg = np.logical_and(sens_attr == group, y_true == 0)
    pos = np.logical_and(sens_attr == group, y_true == 1)
    mean_y0 = np.mean(y_pred[neg])
    mean_y1 = np.mean(y_pred[pos])

    if err > max_error:
      max_error = err
    if err < min_error:
      min_error = err

    if mean_y > max_mean_y:
      max_mean_y = mean_y
    if mean_y < min_mean_y:
      min_mean_y = mean_y

    if mean_y0 > max_mean_y0:
      max_mean_y0 = mean_y0
    if mean_y0 < min_mean_y0:
      min_mean_y0 = mean_y0

    if mean_y1 > max_mean_y1:
      max_mean_y1 = mean_y1
    if mean_y1 < min_mean_y1:
      min_mean_y1 = mean_y1
  
  eo = 0.5*(max_mean_y0 - min_mean_y0 + max_mean_y1 - min_mean_y1)
  dp = max_mean_y - min_mean_y
  err_parity = max_error - min_error

  return eo, dp, err_parity

In [ ]:
#@title Evaluate the model on test data
kwargs = {'verbose': False}
num_batches = 20
metric = tf.keras.metrics.AUC
scores = clf.score(test_data, num_batches, metric, **kwargs)
scores

In [ ]:
yt, yp = clf.predict_mult(test_data, num_batches, **kwargs)
eo, dp, ep = fairness_metrics(yp[0]>=threshold,
                              yt[0],yt[1].astype('int32'))

In [ ]:
#@title Evaluate model on counterfactual test data
kwargs = {'verbose': False}
num_batches = 20
metric = tf.keras.metrics.AUC

scores_flipped = clf.score(test_data_flipped, num_batches,metric, **kwargs)
scores_flipped

In [ ]:
#@title Evaluate attribute encoding based on a frozen feature extractor from the multi-task model
enc = SingleHead(cfg, main="a", dtype=tf.float32, feat_extract=clf.feat_extract)
kwargs = {'epochs': 30, 'steps_per_epoch':20, 'verbose': False}
enc.fit(train_data, **kwargs)
kwargs = {'verbose': False}
num_batches = 20
metric = tf.keras.metrics.AUC
sc = enc.score(test_data, num_batches, metric, **kwargs)
sc

# Piecing the elements together: ShorT

In [ ]:
#@title ShorT method
def shortcut_testing(cfg, train_data, test_data, counterfact_data, val_data,
                     range_grads, seeds = [0,1,2,3,4], num_epochs_train = 30,
                     num_batch_test = 5):
  """Shortcut testing.

  Requires:
  - cfg: a config dictionary of model specifications
  - train_data: tf dataset of train data respecting the positions for x,y,a
  - test_data: tf dataset of test data
  - counterfact_data: tf dataset of counterfactual test images
  - val_data: validation data to compute decision threshold
  - range_grads: np array of gradient scalings
  - seeds: list of seed numbers to fix, per gradient scaling
  - num_epoch_train: int, number of epochs for training
  - num_batch_test: number of batches to test the model on

  Outputs:
  scores_y: np array of size (# gradient scalings, # seeds) of test performance on Y
  encoding_a: np array of model's attribute encoding
  equ_odds: np array of equalized odds on test data
  dem_par: similar, demographic parity
  err_par: error parity
  count_fair: counterfactual fairness (computed locally then averaged)
  scores_c: global performance on Y on counterfactual data
  models: tf.Keras.model instances
  """ 


  scores_y = np.zeros((len(range_grads), len(seeds)))
  scores_c = np.zeros((len(range_grads), len(seeds)))
  encoding_a = np.zeros((len(range_grads), len(seeds)))
  equ_odds = np.zeros((len(range_grads), len(seeds)))
  dem_par = np.zeros((len(range_grads), len(seeds)))
  err_par = np.zeros((len(range_grads), len(seeds)))
  count_fair = np.zeros((len(range_grads), len(seeds)))
  models = []
  metric = tf.keras.metrics.AUC
  loss_weight = cfg.attr_loss_weight
  for g, grad_scaling in enumerate(range_grads):
    cfg.model.attr_grad_updates = float(grad_scaling)

    if grad_scaling == 0:  ## baseline model with no other head
      cfg.attr_loss_weight = float(0.0)
    else:
      cfg.attr_loss_weight = float(loss_weight)
    
    for s, seed in enumerate(seeds):
      tf.random.set_seed(seed)
      np.random.seed(seed)
      # instantiate
      clf = MultiHead(cfg, main="y", aux="a", dtype=tf.float32, pos=None)
      # train the multi-head model
      kwargs = {'epochs': num_epochs_train, 'steps_per_epoch':20, 'verbose': False}
      clf.fit(train_data, **kwargs)
      clf.trainable = False

      # estimate attribute encoding by freezing the weights of the feature extractor
      enc = SingleHead(cfg, main="a", dtype=tf.float32, feat_extract=clf.feat_extract)
      kwargs = {'epochs': num_epochs_train, 'steps_per_epoch':20, 'verbose': False}
      enc.fit(train_data, **kwargs)
      kwargs = {'verbose': False}
      encoding_a[g,s] = enc.score(test_data, num_batch_test,
                                  metric, **kwargs)

      # Model performance on test
      scores = clf.score(test_data, num_batch_test, metric, **kwargs)
      scores_y[g,s] = scores[0]

      # from validation data, obtain threshold for decision making
      out_true, out_pred = clf.predict_mult(val_data,
                                            num_batches=num_batch_test,
                                            **kwargs)
      threshold = threshold_at_max_f1_score(out_true[0],out_pred[0])

      # Model statistical fairness metric
      out_true, out_pred = clf.predict_mult(test_data, num_batches=num_batch_test, **kwargs)
      eo, dp, ep = fairness_metrics(out_pred[0]>=threshold,
                                    out_true[0],out_true[1].astype('int32'))
      equ_odds[g,s] = eo
      dem_par[g,s] = dp
      err_par[g,s] = ep
      # global shortcutting
      scores = clf.score(counterfact_data, num_batch_test, metric, **kwargs)
      scores_c[g,s] = scores[0]
      # counterfactual fairness metric
      cf_true, cf_pred = clf.predict_mult(counterfact_data, num_batches=num_batch_test,
                                          **kwargs)
      count_fair[g,s] = np.mean(np.abs(
          (out_pred[0]>=threshold).astype(float) - (cf_pred[0]>=threshold).astype(float)))
      models.append(clf)
      del clf
  return scores_y, encoding_a, equ_odds, dem_par, err_par, count_fair, scores_c, models

In [ ]:
#@title Run ShorT

# Ensure that these parameters cover the attribute encoding from LEB to UEB
range_grads = [-0.09, -0.07, -0.05, -0.03, -0.02, -0.01, -0.005,0.0, 0.005, 0.01, 0.02, 0.03, 0.05,0.07, 0.09]

# This should take some time to run
acc_y, acc_a, eo, dp, ep, cf, counter_y, models = shortcut_testing(cfg, train_data,
                                                      test_data, valid_data,
                                                      test_data_flipped,
                                                      range_grads,
                                                      num_epochs_train=100,
                                                      num_batch_test=20)


In [ ]:
#@markdown Derive the correlation

filt = acc_y >= 0.8  #@param User-defined performance threshold. The goal is to discard trivial models
corr, p = scipy.stats.spearmanr(acc_a[filt],cf[filt])
print(corr)
print(p)


In [ ]:
#@title Plot utils

SMALL_SIZE = 10
MEDIUM_SIZE = 14
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

palette = sns.color_palette('tab20b')

def plot_scale_gradients_encoding(scale_gradients, encoding_a, 
                                  upper_bound, lower_bound):
  """Plots the intervention compared to attribute encoding.

  Inputs:
  scale_gradients: numpy array of scale gradients used
  encoding_a: numpy array of attribute encoding values, as measure by transfer learning
  upper_bound: list or numpy array of maximum attribute encoding
  lower_bound: list or numpy array of minimum attribute encoding
  """
  
  sg_mean = np.mean(encoding_a, axis=1)
  sg_std = np.std(encoding_a, axis=1)

  fig = plt.figure(figsize=(5,4))
  ax = fig.add_axes([0,0,1,1])
  ax.errorbar(scale_gradients, sg_mean, 
                yerr=sg_std, 
                fmt='x', 
                color='tab:blue',
                ecolor='tab:blue')
  plt.hlines(np.mean(upper_bound),np.min(scale_gradients), np.max(scale_gradients),
            colors=[0.4,0.4,0.4],linestyles='dashed')
  plt.hlines(np.mean(lower_bound),np.min(scale_gradients), np.max(scale_gradients),
            colors=[0.4,0.4,0.4],linestyles='dashed')
  ax.set_xlabel('Scale Gradient')
  ax.set_ylabel('Attribute encoding')
  plt.show

def plot_fairness_encoding(encoding_m, fair_m, perf_m, perf_thresh = 0):
  """Plots fairness results vs attribute encoding.
  
  Inputs:
  encoding_m: encoding metric result, as numpy array
  fair_m: fairness metric result, as numpy array
  perf_m: model performance on the output label, as numpy array
  perf_thresh: what minimum performance to consider
  """

  filt = perf_m <= perf_thresh

  fig = plt.figure(figsize=(5,4))
  plt.scatter(encoding_m, fair_m,color=[0.6,0,0.2],alpha=0.5)
  plt.scatter(encoding_m[filt], fair_m[filt],color=[0.2,0.2,0.2])
  plt.xlabel('Attribute Accuracy')
  plt.ylabel('Fairness')
  plt.show()

def point_z_order(c, midpoint):
  deviation = np.zeros_like(c)
  for i in range(c.shape[0]):
    if c[i] > midpoint:
      deviation[i] = (c[i]-midpoint) / (np.max(c)-midpoint)
    else:
      deviation[i] = (midpoint-c[i]) / (midpoint-np.min(c))
  return np.argsort(deviation)

def performance_fairness_age_frontier_plot(encoding_m, fair_m, perf_m,
                                           scale_gradients, cmap='PRGn'):
  
  class MidpointNormalize(mpl.colors.Normalize):
    """
    class to help renormalize the color scale
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

  baseline_models = np.argwhere(np.array(scale_gradients)==0.0)[0]
  if baseline_models.shape[0] == 0:
    baseline_models = int(len(scale_gradients)/2)
  midpoint = encoding_m[baseline_models,:].mean()
  baseline_model_perf = perf_m[baseline_models,:].mean()
  baseline_model_fair = fair_m[baseline_models,:].mean()

  print(f'Baseline model Attribute encoding: {midpoint:.2f}')
  print(f'Baseline model Performance: {baseline_model_perf:.4f}')
  print(f'Baseline model Fairness: {baseline_model_fair:.4f}')

  norm =  MidpointNormalize(midpoint = midpoint)

  attr = encoding_m.flatten()
  z_order = point_z_order(attr, midpoint)

  fair =fair_m.flatten()
  perf = perf_m.flatten()
  fig = plt.figure(figsize=(5,4))
  ax = fig.add_axes([0,0,1,1])
  plt.scatter(fair[z_order], perf[z_order], s=30, c=attr[z_order], cmap=cmap, norm=norm)
  # overplot the baseline models in red
  plt.scatter(fair_m[baseline_models,:], perf_m[baseline_models,:], s=30,
                   color=(0.8, 0.2, 0.2))
  plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),label='Attribute Encoding')
  plt.ylabel('Performance')
  plt.xlabel('Fairness')
  plt.show()

In [ ]:
#@title Plot scale gradient vs age encoding
plot_scale_gradients_encoding(range_grads, acc_a, upper_bound, lower_bound)

In [ ]:
#@title Plot Fairness w.r.t. attribute encoding
suffix = "{}_EO".format(cfg_data.corr_a_y1)
plot_fairness_encoding(acc_a, cf, acc_y, perf_thresh = 0.8)

In [ ]:
#@title Pareto plot of fairness-performance colored by attribute encoding
performance_fairness_age_frontier_plot(acc_a,eo,acc_y,range_grads)

In [ ]:
#@title Selecting a model and dropping the attribute head for deployment

fairness_threshold = 0.09 #@param
gradient_parameters = np.array(range_grads).reshape(-1,1).repeat(5,axis=1)

possible_models = acc_y[eo<=fairness_threshold]
possible_gradients = gradient_parameters[eo<=fairness_threshold]
filt = eo<=fairness_threshold

ind = np.argmax(possible_models)
selected_gradient = float(possible_gradients[ind])
selected_model = models[np.argwhere(filt.flatten()).tolist()[ind][0]]
print("Selected gradient parameter: {}".format(selected_gradient))
print("Selected model performance: {}".format(possible_models[ind]))
print("Selected model fairness: {}".format(eo[eo<=fairness_threshold][ind]))

In [ ]:
selected_model.model.summary()

In [ ]:
# Create a new model by selecting all layers except the ones related to the attribute
remove = ['attribute', 'attr_branch', 'gradient']  # corresponds to the names of the layer (partial matches ok), as defined in the MultiHead class

final_model = tf.keras.Sequential()
for layer in clf.model.layers:
  match = [to_pop in layer.name for to_pop in remove]
  if not any(match):
    final_model.add(layer)
    layer.trainable = False


In [ ]:
final_model.summary()

In [ ]:
x = test_dataset['train']['image']
y_pred = final_model.predict_on_batch(x)

In [ ]:
sklearn.metrics.roc_auc_score(test_dataset['train']['label'], y_pred)